# Fine-tune

Чтобы улучшить точность predictions основной модели: обучим на тренировочном наборе данных корректирующию полносвязную нейронную сеть (FNN) совместно в основной моделью (Chronos Bolt base).

Корректирующая FNN будет получать на вход выход основной модели и выдавать корректирующие числа в пределах (-1; 1), которые будут умножаться на выбранный коэффициент (для массштабирования коррекций) и прибавляться к единице. В результате будут вычисляться коэффициенты для умножения точек выхода основной модели.

## Сгенерируем тренировочный набор данных для FNN

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import random
import warnings

import numpy as np
import pandas as pd
import torch
from etna.datasets import TSDataset
from etna.metrics import R2, SMAPE
from etna.models.nn import ChronosBoltModel
from etna.pipeline import Pipeline
from tqdm import tqdm

warnings.filterwarnings("ignore")

## Определения

In [3]:
def load_df(path: str):
    """Load data and return as dataframe"""
    df = pd.read_csv(path, usecols=['date_time', 'traffic_volume'])
    df = df.reset_index()
    df = df.drop(columns=['index'])
    return df


def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def df_to_etna_ts(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """Convert df to ETNA format"""
    df["segment"] = "metro"
    df_processed = df.rename(
        columns={"date_time": "timestamp", "traffic_volume": "target"}
    )
    return TSDataset(df=TSDataset.to_dataset(df_processed), freq="h")

Инициализация

In [4]:
N_TRAIN = 84  # Объём тренировки FNN
START_POS = 23590
PERIOD = 24 * 7
HORIZON = PERIOD * 2
TRAIN_SIZE = 24 * 90  # Объём тренировки трансформерной модели (в prompt)

metrics = [SMAPE(), R2()]

set_seed(1)

path_to_chronos_bolt_model = "H:\\Инструменты\\Windows\\GPT or another LLM\
\\amazon chronos-bolt-base 2024"

## Dataset

In [5]:
df_train = load_df("Metro_Interstate_Traffic_Volume_train.csv")
print(df_train)

                 date_time  traffic_volume
0      2016-01-01 01:00:00            1550
1      2016-01-01 02:00:00            1134
2      2016-01-01 03:00:00             719
3      2016-01-01 04:00:00             533
4      2016-01-01 05:00:00             586
...                    ...             ...
23753  2018-09-16 18:00:00            3701
23754  2018-09-16 19:00:00            3400
23755  2018-09-16 20:00:00            3092
23756  2018-09-16 21:00:00            2623
23757  2018-09-16 22:00:00            1725

[23758 rows x 2 columns]


## Получаем тренировочный набор данных для FNN

В контекст Chronos Bolt будут помещаться данные за квартал.<br />
Будем генерировать данные по две недели начиная с воскресения 23:00 (как в тестовом наборе данных).<br />
Смещать данные будем всё дальше в прошлое по целому числу недель, чтобы начало inference всегда было как в тестовом наборе данных: 23:00 воскресения.

Правая граница диапазона выбрана так, чтобы не пересекаться с тестовым набором данных (чтобы исключить утечку тестовых данных в тренировку).

In [6]:
model_chronos_bolt = ChronosBoltModel(
    path_or_url=path_to_chronos_bolt_model,
    encoder_length=TRAIN_SIZE
)

pipeline_etna_bolt = Pipeline(
    model=model_chronos_bolt,
    horizon=HORIZON,
    transforms=[]
)

In [9]:
def infer_iter(df: pd.core.frame.DataFrame,
               start_pos: int,
               n: int,
               size_train: int,
               size_infer: int,
               period: int,
               pipeline_etna: Pipeline):
    """Execute infer by size_infer period."""
    i = start_pos - n * period - size_train - 1
    ts = df_to_etna_ts(df.loc[i:i+size_train-size_infer])
    ts_predict = pipeline_etna.forecast(ts)
    df_predict = ts_predict.df
    y_predict_as_list = list(df_predict["metro"]["target"])
    y_true = df.loc[i+size_train-size_infer+1:i+size_train]
    y_true_as_list = list(y_true["traffic_volume"])
    return y_predict_as_list, y_true_as_list, df_predict

In [ ]:
def do_infer(df: pd.core.frame.DataFrame,
             start_pos: int,
             size_train: int,
             size_infer: int,
             period: int,
             pipeline_etna: Pipeline,
             name_of_output_file: str):
    """Run inference and save to a file"""
    with open(name_of_output_file, "wt") as file_train:
        file_train.write("predict;test\n")

        for i in tqdm(range(N_TRAIN)):
            y_predict, y_true, _ = infer_iter(
                df,
                start_pos,
                i,
                size_train,
                size_infer,
                period,
                pipeline_etna
            )
            file_train.write(str(y_predict)+";"+str(y_true)+"\n")

Проверим, что правая граница тренировки не пересекается с тестовым набором данных.

In [10]:
_, _, control_data = infer_iter(
    df_train,
    START_POS,
    0,
    TRAIN_SIZE,
    HORIZON,
    PERIOD,
    pipeline_etna_bolt
)
print(control_data)

segment                    metro
feature                   target
timestamp                       
2018-08-26 23:00:00  1239.717773
2018-08-27 00:00:00   697.261475
2018-08-27 01:00:00   439.586182
2018-08-27 02:00:00   357.514160
2018-08-27 03:00:00   383.107422
...                          ...
2018-09-09 18:00:00  3816.564209
2018-09-09 19:00:00  3352.195068
2018-09-09 20:00:00  2953.060791
2018-09-09 21:00:00  2687.598877
2018-09-09 22:00:00  2126.250977

[336 rows x 1 columns]


Мы видим, что крайнее правое значение prediction заканчивается 2018-09-09, а тестовый набор данных начинается с 2018-09-16 — утечка тестовых данных в тренировку отсутствует.

Выполним вычисление и сохранение тренировочных данных для FNN.

In [22]:
do_infer(df_train,
         START_POS,
         TRAIN_SIZE,
         HORIZON,
         PERIOD,
         pipeline_etna_bolt,
         "fnn_train.csv")

100%|██████████| 84/84 [00:53<00:00,  1.58it/s]


Мы сохранили тренировочный dataset для FNN в файл `fnn_train.csv`.